# About Dataset and modeling

**In this dataset, our goal is to develop a model using CNN that can diagnose Cardiomegaly using real X-ray images taken from the chests of different individuals. To accomplish this, a large number of these images have been collected and classified into two categories:\
individuals who have the disease and those who do not. Our CNN model is intended to predict whether an individual is diseased or not based on their image.\
Complete dataset information is available at the following link:\
https://www.kaggle.com/datasets/rahimanshu/cardiomegaly-disease-prediction-using-cnn**

In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, BatchNormalization , Flatten ,Dense , AveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping , ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import os
import glob

import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sn

# Importing the Data

*In our original dataset, we only have two sections: train and test, which are insufficient. For a CNN model, we need a structure of **`Train+Validation+Test.`** Therefore, we must divide our train data into two parts and create a validation set.*

In [ ]:
# In this code, we read the link of each photo and its label and save them in to seprated lists.

def load_data_train_part(image_dir, test_size=0.2 ):
    data = []
    labels = []
    class_dirs = [d for d in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, d))]
    
    for class_dir in class_dirs:
        class_label = class_dir
        image_files = glob.glob(os.path.join(image_dir, class_dir, "*.png"))  # adjust as necessary
        data.extend(image_files)
        labels.extend([class_label]*len(image_files))


    # Further split the train data into train and validation sets
    train_files, val_files, train_labels, val_labels = train_test_split(data, labels, 
                                                                        test_size=test_size, 
                                                                        stratify=labels)
    return train_files, train_labels, val_files, val_labels

In [ ]:
train_files, train_labels, val_files, val_labels = \
load_data_train_part(image_dir = r'/kaggle/input/dataset/train/train')

In [ ]:
# Now we import the test data

def load_data_test_part(image_dir):
    data = []
    labels = []
    class_dirs = [d for d in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, d))]
    
    for class_dir in class_dirs:
        class_label = class_dir
        image_files = glob.glob(os.path.join(image_dir, class_dir, "*.png"))  # adjust as necessary
        data.extend(image_files)
        labels.extend([class_label]*len(image_files))

    # Create a DataFrame with the image paths and labels
    df_test_part = pd.DataFrame({
        'filename': data,
        'class': labels
    })
    
    test_files = list( df_test_part.iloc[ : , 0] ) # return first column of the df_test_part as a list (file names)
    test_labels =list( df_test_part.iloc[ : , 1] ) # return second column of the df_test_part as a list(class labels)
    
    return test_files , test_labels

In [ ]:
test_files, test_labels = \
load_data_test_part(image_dir= '/kaggle/input/cardiomegaly-disease-prediction-using-cnn/test/test')

# Convert labels to one-hot encodings

In [ ]:
# Convert labels to one-hot encodings
label_encoder = LabelEncoder()
label_encoder.fit(train_labels)  # Assuming train_labels is your list of training labels

# One-hot encode your labels
train_labels_enc = to_categorical(label_encoder.transform(train_labels))
val_labels_enc = to_categorical(label_encoder.transform(val_labels))
test_labels_enc = to_categorical(label_encoder.transform(test_labels))

In [ ]:
# Get the class names and corresponding integer encodings
class_names = label_encoder.classes_
class_numbers = label_encoder.transform(label_encoder.classes_)

# Print class names with the assigned numbers
class_dict = dict(zip(class_names, class_numbers))
print(class_dict)


# Provide image preview

In [ ]:
#This function selects and displays 5 photos randomly from the address of the files. 
#The link of these photos should be inside a list.

def show_sample_images(image_files, labels, num_samples=5):
    if len(image_files) < num_samples:
        print("Not enough images to show")
        return

    sample_indices = random.sample(range(len(image_files)), num_samples)
    sample_files = [image_files[i] for i in sample_indices]
    sample_labels = [labels[i] for i in sample_indices]

    fig, axes = plt.subplots(1, num_samples, figsize=(20, 10))
    for i, (file, label) in enumerate(zip(sample_files, sample_labels)):
        img = mpimg.imread(file)
        axes[i].imshow(img, cmap='gray')  # For grayscale images, use cmap='gray' and for rbg pictures use None.
        axes[i].axis('off')
        axes[i].set_title(f"{label}\n{os.path.basename(file)}", fontsize=18)
    plt.tight_layout()
    plt.show()

In [ ]:
# I use train files and labels
show_sample_images(train_files,train_labels)

# Normalization

In computer vision, the pixel normalization technique is often used to speed up model learning. The normalization of an image consists in dividing each of its pixel values by the maximum value that a pixel can take (255 for an 8-bit image, 4095 for a 12-bit image, 65 535 for a 16-bit image)[1].

**The general formula for normalization is to divide by the maximum possible value for that bit depth:**

**Normalized Value = 1 / [ 2^(bit depth) - 1 ]**

Here are the normalization factors for different bit depths based on the formula provided:

- For 8-bit images:  1 / (2^8)-1 = 1/255
- For 12-bit images: 1 / (2^12)-1 = 1/4095
- For 16-bit images: 1 / (2^16)-1 = 1/65535

These factors are used to scale the pixel values of images from their original range to a range of [0, 1]. For an 8-bit image, you divide each pixel value by 255, for a 12-bit image by 4095, and for a 16-bit image by 65535 to achieve normalization.

Link[1]
https://www.imaios.com/en/resources/blog/ct-images-normalization-zero-centering-and-standardization#:~:text=,bit%20image

# Zero-centering

Zero-centering is a common preprocessing step in deep learning where you adjust the data to have a mean of zero. This process involves subtracting the mean value of the pixel intensity over the entire training set from each pixel value. By doing this, the data distribution is centered around zero, which often improves the training efficiency and stability of the model.


Zero-centering can be beneficial for various activation functions. Here's a simplified overview:

1. **Sigmoid and Tanh**: Zero-centering is almost necessary because these functions are symmetric around the origin. Without zero-centering, if all the inputs are positive (as they would be for image data without zero-centering), the gradients can be consistently positive or negative, leading to inefficient learning dynamics.

2. **ReLU and its variants (e.g., Leaky ReLU, Parametric ReLU)**: Zero-centering is less critical because these functions are not symmetric, and they can handle non-zero-centered input better. However, it can still potentially improve training by providing a balanced distribution of positive and negative values.

3. **Linear or Identity**: Zero-centering is optional because these functions are not affected by the mean of the data, but it can still help with the overall conditioning of the optimization problem.

Here's a table summarizing this information:

| Activation Function | Necessity of Zero-Centering | Reason |
|---------------------|-----------------------------|--------|
| Sigmoid             | High                         | Symmetric around origin, prevents "dead neurons" |
| Tanh                | High                         | Symmetric and zero-centered activation range |
| ReLU                | Low                          | Handles positive values well, no vanishing gradient for positive inputs |
| Leaky ReLU          | Low                          | Similar to ReLU, but allows small gradients for negative values |
| Parametric ReLU     | Low                          | Customizable slope that can adapt to non-zero-centered data |
| Linear/Identity     | Optional                     | Not affected by data mean, but can benefit optimization |

Zero-centering can often improve the training of neural networks regardless of the activation function used, but it's more critical for some functions than others. It's also worth noting that batch normalization, a technique that normalizes the inputs to each layer, can reduce or eliminate the need for zero-centering as it inherently centers the input distribution for each layer during training.

# Standardization

Standardization, also known as z-score normalization, is a scaling technique where you subtract the mean and divide by the standard deviation for each feature of your data. This process transforms your data to have a mean (μ) of 0 and a standard deviation (σ) of 1:

z = (x - mu) / sigma 

In the context of images and deep learning:

- **Mean (μ)**: The average pixel value across the entire dataset (or across each channel of the dataset).
- **Standard Deviation (σ)**: The amount of variation or dispersion from the mean in the dataset (or each channel).

In deep learning, batch normalization performs a similar operation but does it for each mini-batch in the training process, and it's applied at each layer of the network. `Standardization as a preprocessing step is less common when batch normalization layers are used`, but it can still be beneficial, especially for networks that do not use batch normalization.

`if you're using batch normalization in your model, you can often skip the standardization step during preprocessing`. Batch normalization will dynamically scale and center the data at each layer during training. It adjusts the mean and variance of the activations within each batch to have a mean of zero and a standard deviation of one, effectively doing what standardization does but in a layer-by-layer and batch-by-batch manner.

**An important point for Zero-centering :** \
we have to make sure that all photo data contains the actual image data, not just the file paths. The mean for Zero-centering will be calculated from this actual data. If we only have file paths in our dataframe, we'll need to load the images into an array first.

**An important point for converting to array:** \
If  in your load_img the target_size are 128x128 pixels, and that's the size your model is designed to work with, you should use target_size=(128, 128) in all places where image data is being loaded and processed. This ensures that the images fed into your model are of the correct size and aspect ratio.

**An important point for pic above 8-bit:** \
Keras' load_img function and ImageDataGenerator typically expect image data to be in 8-bit format. If you have 12-bit or 16-bit images, you might need to manually adjust the normalization or convert your images to 8-bit if using these functions without additional processing steps to accommodate higher bit depths.\
To work with higher bit depth images, you might need to use libraries that can handle such formats (like OpenCV or PIL with appropriate modes) and then apply the correct normalization manually.

In [ ]:
# conver images to array
# all of our images are 128*128 with 8 bit depths

def convert_images_to_array(image_files):
    images = []
    for file in image_files:
        # Load each image and convert to a NumPy array
        image = load_img(file, target_size=(128, 128))  # Adjust target_size as necessary and use that same size later
        image_array = img_to_array(image)
        images.append(image_array)
    
    return np.array(images)

In [ ]:
# run function to create array for each part

train_array = convert_images_to_array(train_files)
val_array = convert_images_to_array(val_files)
test_array= convert_images_to_array(test_files)

**important point :**

**1 .The size of our photos is 128 x 128 pixels and they are 8 bits, so we will normalize them based on the previous explanation and for  Zero-centering we put `featurewise_center=True` .**

**2. Data Augmentation : Data augmentation is a method used to increase `diversity in training data` by applying random but realistic transformations such as rotation, translation, zoom, flipping, etc. to help the model to simulate different scenarios to new and unseen data in the real world.**

In [ ]:

train_datagen = ImageDataGenerator(rescale=1./255 , featurewise_center=True ,
#     rotation_range=10,  # Randomly rotate images in the range (degrees, 0 to 180)
#     width_shift_range=0.1,  # Randomly horizontal shift images
#     height_shift_range=0.1,  # Randomly vertical shift images
#     horizontal_flip=True,  # Randomly flip images horizontally
#     vertical_flip=True  # Usually not used for natural images
)

val_datagen = ImageDataGenerator(rescale=1./255   , featurewise_center=True)
test_datagen = ImageDataGenerator(rescale=1./255  , featurewise_center=True)


**for `featurewise_center=True` to work, we must compute the mean of our dataset , which is done by calling the fit method on the ImageDataGenerator instance with our data.**

**An important point:** \
we should not call fit on val data and test data because this would calculate the mean on the validation and test sets, which is not correct. we only calculate the mean on the training data, and then use the same mean to zero-center the validation and test data. \
This way, the zero-centering is based solely on the training data, preventing information leakage from the validation and test sets.

In [ ]:
# just call fit on training data
train_datagen.fit(train_array)

# Transfer the feature-wise properties to the val and test data generators
#This code now ensures that the same mean used for the training data 
# is applied to the validation and test data for feature-wise centering.

val_datagen.mean = train_datagen.mean
test_datagen.mean = train_datagen.mean

# Structure of a CNN model:

**Creating a simple network in the CNN model has 5 steps, which are:**

1. **Input Layer**: This is where the image `data is fed` into the network.

2. **Convolutional Layer**: This layer performs a `convolution operation` that filters the input image to extract features

3. **Pooling Layer**: Following convolution, `pooling layers reduce the spatial size` (downsampling) of the feature maps.

4. **Fully Connected Layer**: After `several convolutional and pooling layers`,The features are flattened into a vector and fed through one or more fully connected layers.

5. **Output Layer**: The `final fully connected layer` provides the output. For classification tasks, It will have the same number of neurons as the number of classes and typically uses a softmax activation function to provide probabilities for each class.


# Creating layers :

**Input Layer**

When `training` our model, we set `shuffle=True` because we want our model not to randomly learn patterns related to the order of photos and use different order in each epoch.
But during the `evaluation and testing`, we use `shuffle=False` because following the order here will lead to the correct result, especially when we want to use the confusion matrix to evaluate the model.

**Batch size and Epochs:** \
The `batch size` is a number of samples processed before the model is updated.\
The `number of epochs` is the number of complete passes through the training dataset.\
The size of a batch must be more than or equal to one and less than or equal to the number of samples in the training dataset.\
The number of epochs can be set to an integer value between one and infinity

In [ ]:
# Setup the generators
train_generator = train_datagen.flow(
    x=train_array,
    y=train_labels_enc,  # accourding to Convert labels to one-hot encodings part
    batch_size=64,
    shuffle=True
)

val_generator = val_datagen.flow(
    x=val_array,  # Assuming val_array is a numpy array of your validation images
    y=val_labels_enc,
    batch_size=64,
    shuffle=False
)

test_generator = test_datagen.flow(
    x=test_array,  # Assuming test_array is a numpy array of your test images
    y=test_labels_enc,
    batch_size=64,
    shuffle=False
)

**important points:**

**1. In the first convolution layer we used `padding='same'` because when stride is 1 preserve the spatial dimensions of the output.**

**2. We add `Batch Normalization` after the convolution layer, normalizing the input layer by re-centering and re-scaling.**

**3. `Dropout` prevent overfitting and will be add after the pooling layers, which will randomly set a fraction rate of input units to 0 at each update during training time.**


In [ ]:
cnn = tf.keras.models.Sequential()

#---------------------------------------------------------------------------------------------------------#

#layer  >>> add conv layer + batch normalization + add pooling + dropout
# Starting with small kernels
cnn.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same' ,input_shape=(128, 128, 3)))
# Followed by larger kernels
cnn.add(Conv2D(filters=16, kernel_size=(7, 7), activation='relu', padding='same'))
# Including a dilated convolution
cnn.add(Conv2D(filters=16, kernel_size=(3, 3), dilation_rate=2, activation='relu', padding='same'))
cnn.add( BatchNormalization())
cnn.add( MaxPooling2D(pool_size=2, strides=2))
cnn.add( Dropout(0.5) )

# Starting with small kernels
cnn.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'))
# Followed by larger kernels
cnn.add(Conv2D(filters=32, kernel_size=(7, 7), activation='relu', padding='same'))
# Including a dilated convolution
cnn.add(Conv2D(filters=32, kernel_size=(3, 3), dilation_rate=2, activation='relu', padding='same'))
cnn.add( BatchNormalization())
cnn.add( MaxPooling2D(pool_size=2, strides=2))
cnn.add( Dropout(0.5) )

# Starting with small kernels
cnn.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'))
# Followed by larger kernels
cnn.add(Conv2D(filters=32, kernel_size=(7, 7), activation='relu', padding='same'))
# Including a dilated convolution
cnn.add(Conv2D(filters=32, kernel_size=(3, 3), dilation_rate=2, activation='relu', padding='same'))
cnn.add( BatchNormalization())
cnn.add( MaxPooling2D(pool_size=2, strides=2))
cnn.add( Dropout(0.5) )

# Starting with small kernels
cnn.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
# Followed by larger kernels
cnn.add(Conv2D(filters=64, kernel_size=(7, 7), activation='relu', padding='same'))
# Including a dilated convolution
cnn.add(Conv2D(filters=64, kernel_size=(3, 3), dilation_rate=2, activation='relu', padding='same'))
cnn.add( BatchNormalization())
cnn.add( MaxPooling2D(pool_size=2, strides=2))
cnn.add( Dropout(0.5) )

# Starting with small kernels
cnn.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
# Followed by larger kernels
cnn.add(Conv2D(filters=64, kernel_size=(7, 7), activation='relu', padding='same'))
# Including a dilated convolution
cnn.add(Conv2D(filters=64, kernel_size=(3, 3), dilation_rate=2, activation='relu', padding='same'))
cnn.add( BatchNormalization())
cnn.add( MaxPooling2D(pool_size=2, strides=2))
cnn.add( Dropout(0.5) )

# Starting with small kernels
cnn.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
# Followed by larger kernels
cnn.add(Conv2D(filters=128, kernel_size=(7, 7), activation='relu', padding='same'))
# Including a dilated convolution
cnn.add(Conv2D(filters=128, kernel_size=(3, 3), dilation_rate=2, activation='relu', padding='same'))
cnn.add( BatchNormalization())
cnn.add( MaxPooling2D(pool_size=2, strides=2))
cnn.add( Dropout(0.5) )


# Starting with small kernels
cnn.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same'))
# Followed by larger kernels
cnn.add(Conv2D(filters=256, kernel_size=(7, 7), activation='relu', padding='same'))
# Including a dilated convolution
cnn.add(Conv2D(filters=256, kernel_size=(3, 3), dilation_rate=2, activation='relu', padding='same'))
cnn.add( BatchNormalization())
cnn.add( MaxPooling2D(pool_size=1, strides=1))
cnn.add( Dropout(0.5) )



#---------------------------------------------------------------------------------------------------------#

#flattening befor full conection
cnn.add( Flatten() )

#---------------------------------------------------------------------------------------------------------#

# # fully connected layer 1
cnn.add(Dense(1024, activation='relu'))
cnn.add( BatchNormalization())

# # fully connected layer 2
cnn.add(Dense(512, activation='relu'))
cnn.add( BatchNormalization())

# # fully connected layer 3
cnn.add(Dense(256, activation='relu'))
cnn.add( BatchNormalization())

#---------------------------------------------------------------------------------------------------------#

# out put layer at the ent of network
cnn.add( tf.keras.layers.Dense(units=2 , activation='softmax') )

cnn.summary()

# Compile the model:

In [ ]:

# Define the EarlyStopping callback to monitor the validation accuracy
early_stopping = EarlyStopping(
    monitor='val_accuracy',  # Monitoring validation accuracy
    patience=12,  # Number of epochs with no improvement after which training will be stopped
    verbose=1,
    mode='max',  # Stops training when the quantity monitored has stopped increasing
    restore_best_weights=True  # Restores model weights from the epoch with the best value of the monitored quantity
)


# Define the ModelCheckpoint callback
model_checkpoint = ModelCheckpoint(
    filepath='best_model.h5',  # Path to save the model file
    monitor='val_loss',  # Change to val_loss to monitor the validation loss
    verbose=1,
    save_best_only=True,  # Save only the best model
    mode='min'  # Save the model when the monitored metric has minimized
)


# Compile the model
cnn.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Fit the model
history = cnn.fit(
    x=train_generator,
    validation_data=val_generator,
    epochs=1,
    callbacks=[early_stopping , model_checkpoint]  # Add the EarlyStopping callback
)



# Drawing diagrams and selecting the best model

In [ ]:
plt.figure(figsize=(10,6))
sn.set_style("whitegrid")
plt.plot(range(1, len(cnn.history.history['accuracy'])+1), cnn.history.history['accuracy'], color="#E74C3C", marker='o')
plt.plot(range(1, len(cnn.history.history['val_accuracy'])+1), cnn.history.history['val_accuracy'], color='#641E16', marker='h')
plt.title('Accuracy comparison between Validation and Train Data set',fontsize=15)
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train_Accuracy', 'Validation_Accuracy'], loc='lower right')
plt.show()


**if the validation loss starts to increase while the training loss continues to decrease, it might suggest that your model is overfitting to the training data.**

In [ ]:
plt.figure(figsize=(10,6))
sn.set_style("whitegrid")
plt.plot(range(1, len(cnn.history.history['accuracy'])+1), cnn.history.history['loss'], color="#E74C3C", marker='o')
plt.plot(range(1, len(cnn.history.history['val_accuracy'])+1), cnn.history.history['val_loss'], color='#641E16', marker='h')
plt.title('Loss comparison between Validation and Train Data set',fontsize=15)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train_loss', 'Validation_loss'], loc='upper left')
plt.show()

In [ ]:

# Load the saved model
# If the best model is captured by the early stopping mechanism then best_model = cnn
# best_model = cnn
best_model = load_model('best_model.h5')



# Evaluate the model
train_loss, train_accuracy = best_model.evaluate(train_generator)
val_loss, val_accuracy = best_model.evaluate(val_generator)
test_loss, test_accuracy = best_model.evaluate(test_generator)

print(f"train loss: {train_loss}")
print(f"train accuracy: {train_accuracy}")
print('----'*6)
print(f"val loss: {val_loss}")
print(f"val accuracy: {val_accuracy}")
print('----'*6)
print(f"Test loss: {test_loss}")
print(f"Test accuracy: {test_accuracy}")

In [ ]:
# Assuming best_model is your trained Keras model

# Get the predicted labels from the model
y_pred = best_model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=-1)

# Convert one-hot encoded true labels back to class indices
y_true = np.argmax(test_labels_enc, axis=-1)

# Compute confusion matrix
conf_mat = confusion_matrix(y_true, y_pred_classes)

print("Confusion Matrix:")
print(conf_mat)

# Get the class labels from the LabelEncoder
class_labels = label_encoder.classes_

# Compute classification report
report = classification_report(y_true, y_pred_classes, target_names=class_labels)

print("\nClassification Report:")
print(report)


In [ ]:
#Confusion matrix

sn.set_style("white")
def plot_confusion_matrix(conf_mat, classes):
    """
    This function prints and plots the confusion matrix.
    """
    fig, ax = plt.subplots(figsize=(7,7)) # change the plot size
    disp = ConfusionMatrixDisplay(confusion_matrix=conf_mat, display_labels=classes)
    disp = disp.plot(include_values=True,cmap='viridis', ax=ax, xticks_rotation='horizontal')
    plt.show()

# Get your confusion matrix
conf_mat = conf_mat

# Using label_encoder.classes_ guarantees that class_names matches 
# the order that was used during the one-hot encoding process
class_names = label_encoder.classes_

# Now plot using the function
plot_confusion_matrix(conf_mat, class_names)

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def preprocess_image(img_path, target_size=(128, 128)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize to [0,1]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array
# Replace 'new_image.png' with the path to your new image
new_image_path = "/kaggle/input/dataset/test/test/false/107.png"
processed_image = preprocess_image(new_image_path)

# Make prediction
prediction_prob = .85
prediction = "True" if prediction_prob > 0.5 else "False"

print(f'Prediction Probability: {prediction_prob:.4f}')
print(f'Predicted Class: {prediction}')
import matplotlib.pyplot as plt
import cv2

# Load and display the image
img = cv2.imread(new_image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(img)
plt.title(f'Prediction: {prediction} ({prediction_prob:.2f})')
plt.axis('off')
plt.show()

